In [0]:
✅ Databricks Test Notebook: test_sqlserver_ingest

# ----------------------------------------
# 📦 Import Required Libraries
# ----------------------------------------
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
import builtins

# ----------------------------------------
# 🔐 Retrieve Secrets (from Key Vault Scope)
# ----------------------------------------
dbutils = builtins.dbutils if hasattr(builtins, "dbutils") else None
assert dbutils is not None, "dbutils not available"

jdbc_url = dbutils.secrets.get(scope="databricks-secrets-lv426", key="sql-jdbc-url")
sql_user = dbutils.secrets.get(scope="databricks-secrets-lv426", key="sql-user")
sql_password = dbutils.secrets.get(scope="databricks-secrets-lv426", key="sql-password")

# ----------------------------------------
# ⚙️ JDBC Config
# ----------------------------------------
connection_properties = {
    "user": sql_user,
    "password": sql_password,
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

# ----------------------------------------
# 🧪 Test Query: Read Sample Table
# ----------------------------------------
table_name = "dbo.vendor_compliance"  # Use schema-qualified name
df = spark.read.jdbc(url=jdbc_url, table=table_name, properties=connection_properties)

# Add identifier for traceability
df = df.withColumn("ingestion_type", lit("test_notebook"))

# ----------------------------------------
# 💾 Write to Delta (Bronze)
# ----------------------------------------
output_path = "/Volumes/thebetty/bronze/vendor_compliance_test"

df.write.format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .save(output_path)

# Optional: Display sample
display(df)
